In [1]:
import numpy as np
import pandas as pd
import sys
import random
import progressbar
import torch
import pickle
from mytree import *
from utils import *
from treeUtil import *
import tqdm
import argparse
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_
import functools
from sklearn.linear_model import LogisticRegression
import gensim
from gensim.models import KeyedVectors
import string
from stanfordcorenlp import StanfordCoreNLP
import nltk.tree
from ast import literal_eval
import pptree
import logging

In [2]:
from nltk.tokenize import word_tokenize
import re
import os
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from logger import Logger

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

In [3]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from spacy.pipeline import merge_entities
nlp = en_core_web_sm.load()
nlp.add_pipe(merge_entities)

In [4]:
import copy
from sklearn.utils import shuffle
from sklearn import metrics

In [5]:
# False if in-domain; True if general
proanti = True
w2vec = False
ner = True
blackout = False
balanced = True
undersample = False
non_trainable = True
economic = True

In [6]:
dic = {0:'ProAnti', 1:'General', 2:'NER', 3:'Blackout', 4:'Balance', 5:'Undersample', 6:'Fixed'}
bool_list = [proanti, w2vec, ner, blackout, balanced, undersample, non_trainable]
corpus_path = '../data/new/economic'
namecode = 'nonrelevant_economic_w2v'
for index, bb in enumerate(bool_list):
    namecode += '_'
    if bb:
        namecode += '1'
    else:
        namecode += '0'
    namecode += dic[index]
namecode

'nonrelevant_economic_w2v_1ProAnti_0General_1NER_0Blackout_1Balance_0Undersample_1Fixed'

In [7]:
#For logging
import logging

#Remove all the previous handlers
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

#Create the file for logging purposes -> CHANGE THE LEVEL TYPE TO logging.DEBUG when debugging or finding faults
logging.basicConfig(filename='log_'+namecode+'.log',
                            filemode='a',
                            format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                            datefmt='%m/%d/%Y %I:%M:%S %p',
                            level=logging.INFO)

In [8]:
logger = Logger('./logs_'+namecode)

In [9]:
class RecursiveNN(nn.Module):
    def __init__(self, word_embeddings, vocab, embedSize=300, numClasses=2, beta = 0.3, use_weight = True, non_trainable = non_trainable):
        super(RecursiveNN, self).__init__()
#             if (w2vec):
#                 self.embedding = nn.Embedding(len(vocab), embedSize)
#                 self.embedding.load_state_dict({'weight': w2vec_weights})
        self.embedding = nn.Embedding.from_pretrained(word_embeddings)
        self.embedding.weight.requires_grad = True
        if non_trainable:
            self.embedding.weight.requires_grad = False
        else:
            self.embedding = nn.Embedding(len(vocab), embedSize)
        self.embedding = nn.Embedding(len(vocab), embedSize)
        self.W = nn.Linear(2*embedSize, embedSize, bias=True)
        self.nonLinear = torch.tanh
        self.projection = nn.Linear(embedSize, numClasses, bias=True)
        self.nodeProbList = []
        self.labelList = []
        self.loss = Var(torch.FloatTensor([0]))
        self.V = vocab
        self.beta = beta
        self.use_weight = use_weight
        self.total_rep = None #
        self.count_rep = 0 #
        self.numClasses = numClasses

    def traverse(self, node):
        if node.isLeaf:
            if node.getLeafWord() in self.V:  # check if right word is in vocabulary
                word = node.getLeafWord()
            else:  # otherwise use the unknown token
                word = 'UNK'
            # print(self.V[word],len(self.V),word,(torch.LongTensor([int(self.V[word])])))
            currentNode = (self.embedding(Var(torch.LongTensor([int(self.V[word])]))))
        else: currentNode = self.nonLinear(self.W(torch.cat((self.traverse(node.left),self.traverse(node.right)),1)))
        currentNode = currentNode/(torch.norm(currentNode))

        assert node.label!=None
        self.nodeProbList.append(self.projection(currentNode))
        # print (node.label)
        self.labelList.append(torch.LongTensor([node.label]))
        loss_weight = 1-self.beta if node.annotated else self.beta
        self.loss += (loss_weight*F.cross_entropy(input=torch.cat([self.projection(currentNode)]),target=Var(torch.cat([torch.LongTensor([node.label])]))))

        #
        if not node.isRoot():
            if self.total_rep is None:
                self.total_rep = currentNode.data.clone()
            else:
                self.total_rep += currentNode.data.clone()
            self.count_rep += 1
        #

        return currentNode        

    def forward(self, x):
        self.nodeProbList = []
        self.labelList = []
        self.loss = Var(torch.FloatTensor([0]))
        self.traverse(x)
        self.labelList = Var(torch.cat(self.labelList))
        return torch.cat(self.nodeProbList)

    def getLoss(self, tree):
        nodes = self.forward(tree)
        predictions = nodes.max(dim=1)[1]
        loss = self.loss
        return predictions,loss

    def getRep(self, tree):
        self.count_rep = 0
        self.total_rep = None
        self.nodeProbList = []
        self.labelList = []
        self.loss = Var(torch.FloatTensor([0]))

        root_rep = self.traverse(tree)

        return (torch.cat((root_rep,self.total_rep/self.count_rep),1)).data.numpy().T.flatten()


    def evaluate(self, trees):
            pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(trees)).start()
            n = nAll = correctRoot = correctAll = 0.0
            tp = [1e-2]*self.numClasses
            fp = [1e-2]*self.numClasses
            fn = [1e-2]*self.numClasses
            f1 = [0.]*self.numClasses
            for j, tree in enumerate(trees):
                predictions,_ = self.getLoss(tree.root)
#                     print((predictions.cpu().data).numpy(),(predictions.cpu().data).numpy().shape)
#                     print((self.labelList.cpu().data).numpy(), (self.labelList.cpu().data).numpy().shape)
                correct = ((predictions.cpu().data).numpy()==(self.labelList.cpu().data).numpy())
#                     print(correct)
                correctAll += correct.sum()
                nAll += np.shape(correct.squeeze())[0] if np.size(correct)!=1 else 1 
                correctRoot += correct.squeeze()[-1] if np.size(correct)!=1 else correct[-1]
#                     print(correct.squeeze()[-1] if np.size(correct)!=1 else correct[-1])
#                     print('actual: {}'.format(tree.root.label))
                for i in range(self.numClasses):
                    size = np.size((predictions.cpu().data).numpy())
                    if size!=1:
                        pred = (predictions.cpu().data).numpy().squeeze()[-1]
                        actual = (self.labelList.cpu().data).numpy().squeeze()[-1]
                    else:
                        pred = (predictions.cpu().data).numpy()[-1]
                        actual = (self.labelList.cpu().data).numpy()[-1]
                    if pred==i and actual==i:
                        tp[i]+=1
                    elif pred==i and actual!=i:
                        fn[i]+=1
                    elif pred==i and actual!=i:
                        fp[i]+=1
                n += 1
                pbar.update(j)
#             print(tp,fp,fn)
            for i in range(self.numClasses):
                p =(1.0*tp[i]/(tp[i]+fp[i]))
                r =(1.0*tp[i]/(tp[i]+fn[i]))
                f1[i] = (2*p*r)/(p+r)
            pbar.finish()
            return correctRoot / n, correctAll/nAll, f1

    def eval_sent_lvl(self,trees,clf):
        pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(trees)).start()
        n = nAll = correctRoot = correctAll = 0.0
        X_predict = []
        Y_gold = []
        for j, tree in enumerate(trees):
            tree_rep = model.getRep(tree.root)
            X_predict.append(tree_rep)
            Y_gold.append(tree.root.label)
        acc = clf.score(np.array(X_predict),np.array(Y_gold))
        return acc

## Data Preprocessing

In [10]:
file_trees = 'econ_without_relevance.pkl'

In [11]:
df_train = pickle.load(open("./trees/df_"+file_trees, 'rb'))

In [12]:
all_tokens = set(sum(df_train['tokens'],[]))

word2idx = {}
word2idx['UNK']=0
i = 1
for token in list(all_tokens):
    word2idx[token] = i
    i+=1

idx2word = {v: k for k, v in word2idx.items()}

In [13]:
idx2word

{0: 'UNK',
 1: 'remunerative',
 2: 'path',
 3: 'Global',
 4: 'note',
 5: 'Attempts',
 6: 'legacy',
 7: 'guidance',
 8: 'militants',
 9: 'slogan',
 10: 'record',
 11: 'Notwithstanding',
 12: 'furnish',
 13: 'lot',
 14: 'District',
 15: 'cotton',
 16: 'phone',
 17: 'summing',
 18: 'stood',
 19: 'withdrawals',
 20: 'give',
 21: 'Slamming',
 22: 'care',
 23: 'sides',
 24: 'capitalising',
 25: 'strongly',
 26: 'Emphasising',
 27: 'Raising',
 28: 'These',
 29: 'sluggish',
 30: 'borrowing',
 31: 'opened',
 32: 'seeded',
 33: 'sources',
 34: 'Wednesdays',
 35: 'recalibrating',
 36: 'Flight',
 37: 'Political',
 38: 'bracket',
 39: 'fail',
 40: 'revolution',
 41: 'bastiyaan',
 42: 'post',
 43: 'approved',
 44: 'chairmanship',
 45: 'monitoring',
 46: 'Asked',
 47: 'State',
 48: 'Also',
 49: 'talk',
 50: 'numerous',
 51: 'immensely',
 52: 'discussion',
 53: 'thetoday',
 54: 'old',
 55: 'Punjabi',
 56: 'chargesheet',
 57: 'luxury',
 58: 'Personal',
 59: 'revamped',
 60: 'steps',
 61: 'advocated',
 

In [14]:
if not w2vec:
    print("Using fine tuned embeddings . .")
    if economic:
        if ner:
            print("Loading economic-word2vec-ner")
            unbiased_in_domain_model = KeyedVectors.load('/Users/navreetkaur/MTP/finetune-word2vec/w2v-models/economic-word2vec-ner')
        elif blackout:
            print("Loading economic-word2vec-blackout")
            unbiased_in_domain_model = KeyedVectors.load('/Users/navreetkaur/MTP/finetune-word2vec/w2v-models/economic-word2vec-blackout')
        else:
            print("Loading economic-word2vec")
            unbiased_in_domain_model = KeyedVectors.load('/Users/navreetkaur/MTP/finetune-word2vec/w2v-models/economic-word2vec')
    else:
        if ner:
            print("Loading tech-word2vec-ner")
            unbiased_in_domain_model = KeyedVectors.load('/Users/navreetkaur/MTP/finetune-word2vec/w2v-models/tech-word2vec-ner')
        elif blackout:
            print("Loading tech-word2vec-blackout")
            unbiased_in_domain_model = KeyedVectors.load('/Users/navreetkaur/MTP/finetune-word2vec/w2v-models/tech-word2vec-blackout')
        else:
            print("Loading tech-word2vec")
            unbiased_in_domain_model = KeyedVectors.load('Users/navreetkaur/MTP/finetune-word2vec/w2v-models/tech-word2vec')
    
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('/Users/navreetkaur/MTP/finetune-word2vec/w2v-models/GoogleNews-vectors-negative300.bin', binary=True)

Using fine tuned embeddings . .
Loading economic-word2vec-ner


In [15]:
embed = np.zeros((len(idx2word), word2vec_model.vector_size))

for idx, word in idx2word.items():
    if not w2vec:
        try: 
            embed[idx] = unbiased_in_domain_model[word]
        except KeyError:
            try:
                embed[idx] = word2vec_model[word]
            except:
                embed[idx] = np.random.normal(size=(word2vec_model.vector_size, ))
    else:
        try:
            embed[idx] = word2vec_model[word]
        except:
            embed[idx] = np.random.normal(size=(word2vec_model.vector_size, ))
        
embed = torch.from_numpy(embed)
df_train.head()

,tokens,target
0,"[We, will, take, action, against, those, who, ...",0
1,"[Raje, said, that, the, state, government, in,...",1
2,"[Das, said, ever, since, the, demonetisation, ...",0
3,"[We, are, simply, providing, a, data, bank, of...",1
4,"[It, is, people, s, mandate, for, the, Prime, ...",1


In [16]:
df_train.head()

,tokens,target
0,"[We, will, take, action, against, those, who, ...",0
1,"[Raje, said, that, the, state, government, in,...",1
2,"[Das, said, ever, since, the, demonetisation, ...",0
3,"[We, are, simply, providing, a, data, bank, of...",1
4,"[It, is, people, s, mandate, for, the, Prime, ...",1


In [17]:
df_train.groupby('target').size()

target
0    797
1    797
dtype: int64

In [18]:
embed.shape, len(word2idx)

(torch.Size([4496, 300]), 4496)

In [19]:
def make_tree(text):
#     print(text)
    output = nlp.annotate(text, properties={
        'annotators': 'tokenize,ssplit,pos,depparse,parse',
        'outputFormat': 'json'
    })
    output = literal_eval(output)
    try:
        tree = str(output['sentences'][0]['parse'])
    except:
        print(output,text)
        return
    # print (tree)
    parse_string = ' '.join(str(tree).split())
    # print(parse_string)
    # print ("\n\n")
    tree = nltk.tree.Tree.fromstring(parse_string)
    tree.chomsky_normal_form()
    tree.collapse_unary(collapseRoot=True,collapsePOS=True)
    nt = convertNLTK_tree(tree)
    return nt

def printLabelTree(tree):
    def inorder(node,nnode):
        if node.isLeaf:
            newnode = pptree.Node('H',nnode)
            wnode = pptree.Node(node.word,newnode)
        elif nnode is not None:
            newnode = pptree.Node('H',nnode)
            inorder(node.left,newnode)
            inorder(node.right,newnode)
        elif node.isRoot():
            newnode = pptree.Node('H')
            inorder(node.left,newnode)
            inorder(node.right,newnode)
            return newnode
        return None
    pptree.print_tree(inorder(tree.root,None))

def create_trees_using_df(df):
    tree = []
    for tokens in list(df['tokens']):
        if len(tokens)==0:
            continue
        line = ' '.join(tokens)
        line += '\n'
        tree.append(make_tree(line))
    return tree

def printlabel(root,l):
    if root:
        l.append(root.label)
#         print(root.label)
        if root.left:
            l+=printlabel(root.left,[])
#             print(printlabel(root.left))
        if root.right:
            l+=printlabel(root.right,[])
#             print(printlabel(root.right))
    return l

In [21]:
# nlp = StanfordCoreNLP('http://localhost', port=9000,timeout=90000)

In [22]:
# neutral = create_trees_using_df(df_train[df_train.target == 0])
# anti = create_trees_using_df(df_train[df_train.target == 0])

In [23]:
# non_neutral = create_trees_using_df(df_train[df_train.target == 1])
# pro = create_trees_using_df(df_train[df_train.target == 1])

In [24]:
# neutral_test = create_trees_using_df(df_test[df_test.target == 0])
# non_neutral_test = create_trees_using_df(df_test[df_test.target == 1])
# anti_test = create_trees_using_df(df_test[df_test.target == 0])
# pro_test = create_trees_using_df(df_test[df_test.target == 1])

In [25]:
# fout = open(file_trees,'wb')
# pickle.dump([pro, anti],fout)
# fout.close()
# fout = open(file_trees+"_test",'wb')
# pickle.dump([pro_test, anti_test],fout)
# fout.close()

In [20]:
[pro,anti] = pickle.load(open(os.path.join('./trees',file_trees),'rb'))
[pro_test,anti_test] = pickle.load(open(os.path.join('./trees',file_trees+"_test"),'rb'))

In [21]:
# fout = open(file_trees,'wb')
# pickle.dump([neutral, non_neutral],fout)
# fout.close()
# fout = open(file_trees+'_test','wb')
# pickle.dump([neutral_test, non_neutral_test],fout)
# fout.close()

In [22]:
CUDA=False
def Var(v):
    if CUDA: return Variable(v.cuda())
    else: return Variable(v)
    
trees = []
raw_words = []
vocab = []

# print("Loading trees...")
# [neutral, non_neutral] = pickle.load(open(file_trees,'rb'))

In [23]:
# for neutral_tree in neutral:
#     neutral_tree.root.set_label('neutral')
# for non_neutral_tree in non_neutral:
#     non_neutral_tree.root.set_label('non_neutral')
    
for pro_tree in pro:
    pro_tree.root.set_label('pro')
for anti_tree in anti:
    anti_tree.root.set_label('anti')

In [24]:
# for neutral_tree in neutral_test:
#     neutral_tree.root.set_label('neutral')
# for non_neutral_tree in non_neutral_test:
#     non_neutral_tree.root.set_label('non_neutral')
    
for pro_tree in pro_test:
    pro_tree.root.set_label('pro')
for anti_tree in anti_test:
    anti_tree.root.set_label('anti')

In [25]:
def combine(neutral,non_neutral):
    trees = []
    trees.extend(neutral)
    trees.extend(non_neutral)
    random.shuffle(trees)
    return trees

In [26]:
from utils import *

## utils.py

In [27]:
from mytree import *
from treeUtil import *
# from pycorenlp import StanfordCoreNLP

val = {'pro':0,'anti':1,'default':-1}

# pro:1, anti:0, neutral:2
val_all = {'pro':1,'anti':0,'default':-1,'neutral':2}
# neutral:0, non-neutral:1
val_neutral = {'neutral':0, 'non_neutral':1, 'default':-1}


def convert(T):
    label = val[T.label] if (hasattr(T,'label')) else None
    print(label)
    newTree = convert_primary_new(T,label)
    annotate_all(newTree)

    return newTree

def convert_neutral(T):
    label = val_neutral[T.label] if (hasattr(T,'label')) else None
    print(label)
    newTree = convert_primary_new(T,label)
    annotate_all(newTree)

    return newTree

def convert_all(T):
    label = val_all[T.label] if (hasattr(T,'label')) else None
    print(label)
    newTree = convert_primary_new(T,label)
    annotate_all(newTree)

    return newTree


def convert_primary(T):
    if (hasattr(T,'label')):
        print(T.label) 
    label = val[T.label] if (hasattr(T,'label')) else None
    # label = val[T.label] if (hasattr(T,'label') ) else None # changed for ignoring neutral

    if isinstance(T,leafObj):
        newTree = Node(label,T.word,T.pos)
        newTree.isLeaf = True
        return newTree
    else:
        newTree = Node(label)
    
    leftChild = convert_primary(T.c1)
    rightChild = convert_primary(T.c2)
    leftChild.parent = newTree
    rightChild.parent = newTree

    newTree.left = leftChild
    newTree.right = rightChild

    return newTree

def convert_primary_new(T,label):
    # from IPython import embed; embed()
    if T is None:
        return None
    # label = val[T.label] if (hasattr(T,'label') ) else None # changed for ignoring neutral
    T.set_label(label)
    # if (T.isLeaf) : print (T.word)

    T.left = convert_primary_new(T.left,label)
    T.right = convert_primary_new(T.right,label)

    return T

    # if T.isLeaf:
    #     newTree = Node(label,T.word,T.pos)
    #     newTree.isLeaf = True
    #     return newTree
    # else:
    #     newTree = Node(label)
    
    # leftChild = convert_primary_new(T.left)
    # rightChild = convert_primary_new(T.right)
    # leftChild.parent = newTree
    # rightChild.parent = newTree

    # newTree.left = leftChild
    # newTree.right = rightChild

    # return newTree

def convertNLTK_tree_primary(tree):
    if tree.height()==2:
        newTree = Node('default',tree[0],None)
        newTree.isLeaf = True
        return newTree
    newTree = Node('default')
    leftChild = convertNLTK_tree_primary(tree[0])
    rightChild = convertNLTK_tree_primary(tree[1])
    
    leftChild.parent = newTree
    rightChild.parent = newTree

    newTree.left = leftChild
    newTree.right = rightChild

    return newTree

def convertNLTK_tree(tree):
    return Tree(convertNLTK_tree_primary(tree))




def annotate_all(T):
    if T == None: return
    if T.label != None : 
        T.annotated = True
    else:
        T.annotated = False
        T.set_label(T.parent.label)
    annotate_all(T.left)
    annotate_all(T.right)

def buildBalTree(sent):
    words = sent.split(' ')

    nodes = words

    while len(nodes)>1:
        temp = []
        for i in range(0,len(nodes),2):
            lChild = Node(None,nodes[i],None) if isinstance(nodes[i],str) else nodes[i]
            if i+1<len(nodes):
                rChild = Node(None,nodes[i+1],None) if isinstance(nodes[i+1],str) else nodes[i+1]
            else:
                rChild = None
            if isinstance(nodes[i],str):
                lChild.isLeaf = True
                if rChild is not None:
                    rChild.isLeaf = True
            newNode = Node(None)
            lChild.parent = newNode
            newNode.left = lChild
            newNode.right = rChild
            if rChild is not None:
                rChild.parent = newNode
            temp.append(newNode)
        nodes=temp
    return Tree(nodes[0])

def readFile2Trees(filename):
    trees = []
    with open(filename,'r') as file:
        for line in file:
            if line=='\n':
                continue
            else:
                [labelname,sent] = line.split(': ',1)
                tree = buildBalTree(sent)
                tree.root.set_label(val[labelname])
                if val[labelname]!=2:
                    trees.append(tree)
    return trees

In [28]:
# trees = combine(neutral, non_neutral)
trees = combine(pro,anti)
data = []
for i in range(len(trees)):
    data.append(Tree(convert(trees[i].root)))

0
1
0
1
1
0
1
1
1
1
0
1
0
1
0
0
0
0
1
1
1
1
1
1
1
1
0
1
0
1
0
1
1
0
1
1
1
0
0
0
0
1
1
1
0
1
0
0
1
0
1
0
0
1
0
0
0
1
1
0
1
0
1
1
0
1
1
0
0
1
0
0
1
1
1
0
0
1
0
0
1
0
0
0
0
1
0
1
1
1
1
1
0
0
0
1
1
0
1
0
0
0
1
1
1
1
0
1
1
1
0
1
0
1
0
1
1
0
1
0
1
0
1
1
1
1
1
1
1
0
1
1
0
0
1
0
1
1
1
0
0
1
1
1
0
0
1
1
1
0
0
0
1
1
0
1
1
1
1
1
1
1
0
0
0
1
1
1
1
0
1
0
0
1
1
0
1
0
0
1
1
1
1
0
1
0
1
1
1
1
1
0
0
1
1
0
1
1
0
1
1
1
0
1
0
0
1
1
0
1
0
0
0
0
1
1
0
0
0
0
1
1
1
0
0
1
0
1
0
1
1
1
0
0
1
0
1
0
1
1
0
0
1
0
0
0
1
0
0
1
1
0
1
1
0
1
0
0
0
0
0
0
1
0
0
1
1
0
1
1
1
1
1
1
1
1
0
1
1
0
0
0
0
0
0
0
1
0
0
1
0
1
1
1
1
1
1
0
1
0
1
1
1
0
0
1
0
1
1
1
0
0
1
0
0
0
1
0
1
0
1
1
0
0
0
1
0
1
1
1
0
0
1
0
0
0
0
0
0
1
0
1
1
0
1
0
1
0
0
1
0
1
1
0
0
0
0
0
1
1
1
0
1
1
0
0
0
1
1
1
1
1
1
1
1
0
0
0
1
0
1
1
0
0
1
0
0
0
1
1
0
0
1
1
0
1
0
0
0
1
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
1
0
0
0
0
1
0
1
0
0
0
0
0
0
1
0
0
1
0
1
0
0
0
0
1
0
1
1
1
0
0
1
1
0
0
0
0
0
1
1
0
0
1
1
0
1
0
0
1
1
1
1
1
0
0
0
1
1
0
1
0
0
0
0
0
0
0
1
0
0
0
1
1
0
0
0
0
0
1
0
1
1
1
1
0


In [29]:
# trees_test = combine(neutral_test, non_neutral_test)
trees_test = combine(pro_test,anti_test)
data_test = []
for i in range(len(trees_test)):
    data_test.append(Tree(convert(trees_test[i].root)))

0
0
0
0
1
1
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
1
1
1
0
0
1
0
0
0
0
1
1
0
0
1
1
0
1
0
0
1
0
0
0
0
0
1
0
0
1
0
0
0
1
0
1
0
0
0
1
1
1
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
1
1
0
0
0
1
1
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
1
1
0
0
0
0
1
0
1
0
0
0
0
1
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
1
0
1
1
0
0
0
0
0
1
1
0
1
0
1
0
0
1


In [30]:
if CUDA: model = RecursiveNN(embed,word2idx).cuda()
else: model = RecursiveNN(embed,word2idx)
max_epochs = 100
widgets = [progressbar.Percentage(), ' ', progressbar.Bar(), ' ', progressbar.ETA()]
l2_reg = {  'embedding.weight' : 1e-6,'W.weight' : 1e-4,'W.bias' : 1e-4,'projection.weight' : 1e-3,'projection.bias' : 1e-3}
random.shuffle(data)

In [31]:
# trn,dev = data[:int((len(data)+1)*.85)],data[int(len(data)*.85+1):]
# len(data), len(trn), len(dev)
trn = data
dev = data_test

In [32]:
len(trn), len(dev)

(1594, 192)

In [33]:
logging.info('Start logging')

In [34]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, dampening=0.0)
# bestAll=bestRoot=0.0
global count
count=0
BATCH_SIZE = 128
# optimizer = torch.optim.LBFGS(model.parameters(), lr=0.5, max_iter=10, history_size = 10)
bestAll=bestRoot=0.0
bestF1_0=bestF1_1=best_trn_F1_0=best_trn_F1_1=0.0
best_trn_All = best_trn_Root = 0.0 

In [42]:
for epoch in range(max_epochs):
#     trn = get_trn()
    print("\n\nEpoch %d" % epoch)
    logging.info('Epoch: '+str(epoch))
#     pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(trn)/BATCH_SIZE).start()
    params = []
    for i in range(0,len(trn),BATCH_SIZE):
        count+=1
        batch = trn[i:min(i+BATCH_SIZE,len(trn))]
        def closure():
            global count
            optimizer.zero_grad()
            _,total_loss = model.getLoss(trn[0].root)
            for tree in batch:
                _, loss = model.getLoss(tree.root)
                total_loss += loss

            total_loss = total_loss/len(batch)
            #L2 reg
            param_dict = dict()
            for name, param in model.named_parameters():
                param_dict[name] = param.data.clone()
                if param.requires_grad:
                        total_loss += 0.5*l2_reg[name]*(torch.norm(param)**2)
            params.append(param_dict)
            print('Loss = ',total_loss.data)
            logging.info('Loss = '+str(total_loss.data))
            logger.scalar_summary('loss', total_loss.data, count)
            total_loss.backward()
            clip_grad_norm_(model.parameters(),5,2)
            return total_loss
#         pbar.update(i/BATCH_SIZE)
        optimizer.step(closure)

#     pbar.finish()

    avg_param = dict()
    for name, param1 in model.named_parameters():
            avg_param[name] = param1.data.clone()

    for i in range(1,len(params)):
        for name, param in params[i].items():
            avg_param[name] += param.clone()
    for name, param in model.named_parameters():
        if name == 'embedding.weight':
            continue
        param.data = avg_param[name]/len(params)

    correctRoot, correctAll, f1 = model.evaluate(dev)
    # correctRoot = model.eval_sent_lvl(dev,LR_clf)
    if bestAll<correctAll: bestAll=correctAll
    if bestRoot<correctRoot: bestRoot=correctRoot
    if bestF1_0<f1[0]: bestF1_0=f1[0]
    if bestF1_1<f1[1]: bestF1_1=f1[1]
    print("\nValidation All-nodes accuracy:"+str(round(correctAll,2))+"(best:"+str(round(bestAll,2))+")")
    print("Validation Root accuracy:" + str(round(correctRoot,2))+"(best:"+str(round(bestRoot,2))+")")
    print("F1:"+str([round(x,2) for x in f1])+"(best:"+str(round(bestF1_0,2))+" , "+str(round(bestF1_1,2))+")")
    logging.info("Validation All-nodes accuracy:"+str(round(correctAll,2))+"(best:"+str(round(bestAll,2))+")")
    logging.info("Validation Root accuracy:" + str(round(correctRoot,2))+"(best:"+str(round(bestRoot,2))+")")
    logging.info("F1:"+str([round(x,2) for x in f1])+"(best:"+str(round(bestF1_0,2))+" , "+str(round(bestF1_1,2))+")")
    correct_trn_Root, correct_trn_All, f1_trn = model.evaluate(trn)
    # correctRoot = model.eval_sent_lvl(dev,LR_clf)
    if best_trn_All<correct_trn_All: best_trn_All=correct_trn_All
    if best_trn_Root<correct_trn_Root: best_trn_Root=correct_trn_Root
    if best_trn_F1_0<f1_trn[0]: best_trn_F1_0=f1_trn[0]
    if best_trn_F1_1<f1_trn[1]: best_trn_F1_1=f1_trn[1]
    print("\nTraining All-nodes accuracy:"+str(round(correct_trn_All,2))+"(best:"+str(round(best_trn_All,2))+")")
    print("Training Root// accuracy:" + str(round(correct_trn_Root,2))+"(best:"+str(round(best_trn_Root,2))+")")
    print("Training F1:"+str([round(x,2) for x in f1_trn])+"(best:"+str(round(best_trn_F1_0,2))+" , "+str(round(best_trn_F1_1,2))+")")
    logging.info("Training All-nodes accuracy:"+str(round(correct_trn_All,2))+"(best:"+str(round(best_trn_All,2))+")")
    logging.info("Training Root accuracy:" + str(round(correct_trn_Root,2))+"(best:"+str(round(best_trn_Root,2))+")")
    logging.info("Training F1:"+str([round(x,2) for x in f1_trn])+"(best:"+str(round(best_trn_F1_0,2))+" , "+str(round(best_trn_F1_1,2))+")")
    info = {'valid_root_acc':correctRoot, 'valid_tree_acc':correctAll, 'train_root_acc':correct_trn_Root, 'train_tree_acc':correct_trn_All}
    for tag, value in info.items():
        logger.scalar_summary(tag,value,epoch)
    random.shuffle(trn)
    if epoch>0 and epoch%10==0:
        pickle.dump(model,open("./models/checkpoints/"+namecode+'_epoch_'+str(epoch)+'.pkl','wb'))



Epoch 0
Loss =  tensor([32.8689])
Loss =  tensor([33.5596])
Loss =  tensor([32.9465])
Loss =  tensor([32.6569])
Loss =  tensor([32.6983])
Loss =  tensor([33.3221])
Loss =  tensor([32.8873])
Loss =  tensor([33.3159])
Loss =  tensor([32.4685])
Loss =  tensor([32.0707])
Loss =  tensor([31.8878])
Loss =  tensor([31.2146])
Loss =  tensor([33.5853])
Loss =  tensor([30.7125])
Loss =  tensor([31.5310])
Loss =  tensor([30.8620])
Loss =  tensor([33.5052])
Loss =  tensor([29.9537])
Loss =  tensor([31.3216])
Loss =  tensor([34.8003])
Loss =  tensor([34.2795])
Loss =  tensor([33.8993])
Loss =  tensor([32.4897])
Loss =  tensor([32.4836])
Loss =  tensor([30.9470])
Loss =  tensor([33.2213])


100% |##########################################################| Time: 0:00:10



Validation All-nodes accuracy:0.42(best:0.42)
Validation Root accuracy:0.44(best:0.44)
F1:[0.81, 0.52](best:0.81 , 0.52)


100% |##########################################################| Time: 0:01:30



Training All-nodes accuracy:0.52(best:0.52)
Training Root// accuracy:0.51(best:0.51)
Training F1:[0.69, 0.67](best:0.69 , 0.67)


Epoch 1
Loss =  tensor([34.1436])
Loss =  tensor([31.2487])
Loss =  tensor([31.2259])
Loss =  tensor([33.3744])
Loss =  tensor([30.9114])
Loss =  tensor([34.0492])
Loss =  tensor([33.4215])
Loss =  tensor([32.6466])
Loss =  tensor([31.6466])
Loss =  tensor([31.0408])
Loss =  tensor([34.3197])
Loss =  tensor([34.5765])
Loss =  tensor([29.7594])
Loss =  tensor([33.0984])
Loss =  tensor([32.8109])
Loss =  tensor([32.6613])
Loss =  tensor([33.2787])
Loss =  tensor([32.1721])
Loss =  tensor([33.6185])
Loss =  tensor([32.1197])
Loss =  tensor([33.1291])
Loss =  tensor([32.3764])
Loss =  tensor([34.1489])
Loss =  tensor([29.6337])
Loss =  tensor([31.9362])
Loss =  tensor([35.0261])


...


Training All-nodes accuracy:0.67(best:0.68)
Training Root// accuracy:0.98(best:0.98)
Training F1:[0.99, 0.99](best:0.99 , 0.99)


Epoch 99
Loss =  tensor([23.5220])
Loss =  tensor([24.9179])
Loss =  tensor([23.5576])
Loss =  tensor([25.4810])
Loss =  tensor([23.8800])
Loss =  tensor([25.7025])
Loss =  tensor([23.5086])
Loss =  tensor([26.2647])
Loss =  tensor([26.4396])
Loss =  tensor([25.0910])
Loss =  tensor([25.2757])
Loss =  tensor([24.0033])
Loss =  tensor([26.0681])
Loss =  tensor([27.2316])
Loss =  tensor([23.4788])
Loss =  tensor([26.1439])
Loss =  tensor([28.1391])
Loss =  tensor([26.5988])
Loss =  tensor([25.2074])
Loss =  tensor([25.3713])
Loss =  tensor([26.6978])
Loss =  tensor([25.2580])
Loss =  tensor([25.2057])
Loss =  tensor([26.2146])
Loss =  tensor([25.6738])
Loss =  tensor([25.2007])


100% |##########################################################| Time: 0:00:08



Validation All-nodes accuracy:0.56(best:0.59)
Validation Root accuracy:0.68(best:0.7)
F1:[0.84, 0.7](best:0.85 , 0.73)


100% |##########################################################| Time: 0:01:04



Training All-nodes accuracy:0.68(best:0.68)
Training Root// accuracy:0.98(best:0.98)
Training F1:[0.99, 0.99](best:0.99 , 0.99)


In [35]:
namecode

'nonrelevant_economic_w2v_1ProAnti_0General_1NER_0Blackout_1Balance_0Undersample_1Fixed'

In [36]:
import pickle
pickle.dump(model,open("./models/"+namecode+'.pkl','wb'))